In [30]:
import numpy as np
import pandas as pd
import keras
from keras.applications.vgg19 import VGG19
from keras.applications.vgg19 import preprocess_input
from keras.applications.vgg19 import decode_predictions
from keras.models import Model
from keras.layers import Dense, Dropout, Flatten
import cv2 as cv

display("Loading model....")
model = VGG19(weights='imagenet')
display("Model loaded.")

#display("Preprocessing Image")
#imageName = "ex1.png"
#image = cv.imread(imageName)
#imageHeight, imageWidth, imageChannels = image.shape
#display("Image " + str(imageName) + " is " + str(imageWidth) + "x" + str(imageHeight) + ".")
#image = cv.resize(image, (224,224)).astype(np.float32)
#image = image.transpose((2,0,1))
#image = np.expand_dims(image, axis=0)

#display("Predicting...")
#features = model.predict(image)
#display("Done!")
#label = decode_predictions(features)
#display(label)
#display(label[0])
#display(label[0][0])
#label = label[0]
#display(str(label[0][1]) + " (" + str(label[0][2]*100)+"%)")
#display(str(label[0]) + " (" + str(label[1]*100)+"%)")

'Loading model....'

'Model loaded.'

In [51]:
#Read in image
imageName = "ex1.png"
image = cv.imread(imageName)
imageHeight, imageWidth, imageChannels = image.shape

#Parameters to play around with
windowSize = 224     #REMEBER TO CHANGE THIS TO 128!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

numStacks = 4       #Nunmber of times to shrink the image. If this shrinks the image past
                    #the point where the image size is less than the window size, the script will end
                    #without any loss
        
zoomFactor = 2     # How much to initially enlarge image in order to zoom in on smaller boxes.
                   # Smallest box size wikll be windowSize / zoomFactor, so a zoomfactor of 2 will
                   # have a minimum size of 112 with a window size of 224.
        
scaleFactor = .9  #Determines how much smaller each subsequent scaling of the image is. Percentage

scaleX = 1         # How many times to apply a scale in the x direction only

scaleY = 1        # How many times to apply a scale in the y direction only

scaleXFactor = 1.5   # How much to stretch when scaling in the x direction (Percentage)

scaleYFactor = 1.5  # How much to stretch when scaling in the y direction

convStrideFactor = .2 # Percent of window size to use as convStride
convStride = int(windowSize * convStrideFactor)

class boxResult:
    x=0
    y=0
    height=0
    width=0
    classification=0
    confidence=0.0

def modelPredict(predictionImage, currentScale, currentScaleX=1, currentScaleY=1):
    predictionImage = predictionImage.astype(np.float32)
    predictionImage = np.expand_dims(predictionImage, axis=0)
    result = model.predict(predictionImage)

    ######################################################
    label = decode_predictions(result)[0]
    display(label)

    #This section makes sense of the VGG19 results and makes sense of them
    #As long as the highest weighted result ends up in temp.classification
    #and temp.confidence below, everything should still work when switching
    #models.
    ######################################################
    #Store results in an ar
    #x, y, h, w, class, confidence
    temp = boxResult()
    temp.x = int(x / currentScaleX / currentScale)
    temp.y = int(y / currentScaleY / currentScale)
    temp.height = int(windowSize / currentScaleX / currentScale)
    temp.width = int(windowSize / currentScaleY / currentScale)
    temp.classification = label[1]################################
    temp.confididence = label[2]##################################
    return temp

#Image info
display("Image " + str(imageName) + " is " + str(imageWidth) + "x" + str(imageHeight) + ".")



#Scale up image to get zoomed in squares
image = cv.resize(image, None, fx=zoomFactor, fy=zoomFactor, interpolation = cv.INTER_CUBIC)
currentScale = zoomFactor
results = []

# Get frames then scale down
# number of times based on numStacks


# Iterate over the total number of stacks
for i in range (0,numStacks): 
    display("Stack #"+str(i))
    
    # Iterate over number of vertical strides 
    # (From top of image to the bottom)
    #
    for y in range (0, int(imageHeight * currentScale) - windowSize):
        
        print('y: ', y)
        
        
        # !!! Not sure what this case actually is..... 
        #     MAY NOT BE A FUNCTIONAL CHECK
        #
        if y % convStride == 0:
            
            
            # Iterate image horizontally 
            #
            for x in range (0, int(imageWidth * currentScale) - windowSize):
                
                # !!! Also unsure what this case actually means
                #     MAY NOT BE A FUNCTIONAL CHECK
                #
                if x % convStride == 0:
                    
                    #Perform classification on the baseline window
                    crop = image[y:y+windowSize, x:x+windowSize]
                    temp = modelPredict(crop, currentScale)
                    results.append(temp)
                    
                    #Perform classification on any stretched windows
                    #X Stretches
                    for numStretches in range (0, scaleX):
                        display("Scaling x")
                        stretchedCrop = cv.resize(crop, None, fx=scaleXFactor*(1+numStretches), fy=1, interpolation = cv.INTER_CUBIC)
                        stretchedCropCropped = stretchedCrop[0:windowSize, 0:windowSize]
                        temp = modelPredict(stretchedCropCropped, currentScale, scaleXFactor*(1+numStretches))
                        results.append(temp)
                    for numStretches in range (0, scaleY):
                        display("Scaling y")
                        stretchedCrop = cv.resize(crop, None, fx=1, fy=scaleYFactor*(1+numStretches), interpolation = cv.INTER_CUBIC)
                        stretchedCropCropped = stretchedCrop[0:windowSize, 0:windowSize]
                        temp = modelPredict(stretchedCropCropped, currentScale, 1, scaleYFactor*(1+numStretches))
                        results.append(temp)
                    
    #Resize image, update currentScale
    image = cv.resize(image, None, fx=scaleFactor, fy=scaleFactor, interpolation = cv.INTER_AREA)
    currentScale *= scaleFactor
    
display("Done!")

'Image ex1.png is 602x404.'

'Stack #0'

y:  0


[('n06359193', 'web_site', 0.24174668),
 ('n03532672', 'hook', 0.066418774),
 ('n03291819', 'envelope', 0.04223133),
 ('n04118776', 'rule', 0.031054791),
 ('n03126707', 'crane', 0.0205233)]

'Scaling x'

...

'Scaling x'

[('n01665541', 'leatherback_turtle', 0.07564131),
 ('n09399592', 'promontory', 0.051776387),
 ('n09428293', 'seashore', 0.040909637),
 ('n09246464', 'cliff', 0.03278605),
 ('n02037110', 'oystercatcher', 0.029541684)]

'Scaling y'

[('n02417914', 'ibex', 0.15534467),
 ('n03743016', 'megalith', 0.07304825),
 ('n02361337', 'marmot', 0.07015143),
 ('n09246464', 'cliff', 0.063645765),
 ('n09399592', 'promontory', 0.062647946)]

KeyboardInterrupt: 

In [32]:
storageImage = cv.imread(imageName)
displayImage = storageImage.copy()
prevHeight = 0
lineColor = (255,0,0)
lineWidth = 2
for box in results:
    cv.rectangle(displayImage,(box.x,box.y),(box.x+box.width,box.y+box.height),lineColor,lineWidth)
    cv.imshow('Windows',displayImage)
    cv.waitKey(500)
    displayImage = storageImage.copy()
cv.destroyAllWindows()

In [45]:
#Variables to play with
confidenceThresh = 0.8 #Poin whether or not to prune a box from the list

#Create copy of results list to make playing with data easier
boxes = list(results)
             
#Pruning
for box in boxes:
    if box.confidence < confidenceThresh:
        boxes.remove(box)

#NMS
def nmsFast(boxes, overlapThreshold):
    #Ensure there are boxes
    if len(boxes) == 0:
        return []
    
    #Initialize list of picked boxes, and of coordinates
    x1 = np.array([])
    x2 = np.array([])
    y1 = np.array([])
    y2 = np.array([])
    pick = []
    
    #Get coordinates from bounding boxes and cast as floats
    for box in boxes:
        np.append(x1, float(box.x))
        np.append(x2, float(box.x+box.width))
        np.append(y1, float(box.y))
        np.append(y2, float(box.y+box.height))
    
    #Compute and store area of bounding boxes
    area = (x2-x1+1) * (y2-y1+1)
    
    #Sort boxes by bottom right coordinates
    idxs = np.argsort(y2)
    
    while len(idxs) > 0:
        last = len(idxs)-1
        i = idxs[last]
        pick.append(i)
        
        xx1 = np.maximum(x1[i],x1[idxs[:last]])
        xx2 = np.minimum(x2[i],x2[idxs[:last]])
        yy1 = np.maximum(y1[i],y1[idxs[:last]])
        yy2 = np.minimum(y2[i],y2[idxs[:last]])
        
        #compute width and height
        w = np.maximum(0, xx2-xx1+1)
        h = np.maximum(0, yy2-yy1+1)
        
        #compute overlap
        overlap = (w*h) / area[adxs[:last]]
        
        #delete indices that have overlap over overlapThreshold
        idxs = np.delete(idxs, np.concatenate(([last], np.where(overlap > overlapThresh)[0])))
    
    
    
    print(pick)
    
    return boxes

In [50]:
nms_rez = nmsFast(results, .9)
print(len(nms_rez))






[]
230
